# 導入

In [19]:
# pip install jupyter
# pip install pandas
# pip install lxml
# pip install tqdm
# pip install selenium
# pip install webdriver-manager

# インポート

In [20]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup # Htmlの細かい要素を取り出したい場合に使うモジュール
import re #正規表現を使いたい場合のモジュール
import time
from tqdm.notebook import tqdm # 実行進捗を可視化できるモジュール

# 【リメーク版】競馬予想で始める機械学習

# データ取得

## 目標  
- 2023年のレース結果のテーブルをスクレイピングにより取得して、Pythonで扱えるようにする

## ステップ  
1. カレンダーのページから開催日一覧を取得（スクレイピング）  
    1. 2023年1月であれば：https://race.netkeiba.com/top/calendar.html?year=2023&month=1  
2. 開催ページからレースid一覧を取得 
    1. 2023年1月5日の開催であれば：https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105 
3. レース結果ページからレース結果テーブル一覧を取得  

### （用語の解説）スクレイピングとは  
- プログラムによって、Webページから自動的にデータを取得すること

In [21]:
from urllib.request import urlopen

url = "https://db.netkeiba.com/race/202505021011/"
html = urlopen(url).read()

In [22]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xcd\xa5\xbd\xd9\xcc\xc6\xc7\xcf\xa1\xc32025\xc7\xaf5\xb7\xee25\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\

In [23]:
import pandas as pd

pd.read_html(html)

[    着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気  \
 0     1    7   15     カムニャック  牝3  55  シュタル  2:25.7    NaN   14.3    4   
 1     2    1    1  アルマヴェローチェ  牝3  55  岩田望来  2:25.7    アタマ    3.4    2   
 2     3    7   13     タガノアビー  牝3  55  藤岡佑介  2:25.9  1.1/4   49.0   10   
 3     4    2    3    パラディレーヌ  牝3  55  丹内祐次  2:26.0     クビ   17.7    7   
 4     5    3    5   リンクスティップ  牝3  55  Ｍ．デム  2:26.2  1.1/4    3.7    3   
 5     6    6   11   ウィルサヴァイブ  牝3  55  団野大成  2:26.4  1.1/4  328.8   18   
 6     7    6   12  ブラウンラチェット  牝3  55   レーン  2:26.6  1.1/2   15.3    6   
 7     8    8   17  ケリフレッドアスク  牝3  55   ディー  2:26.7     クビ  188.7   14   
 8     9    5    9   エンブロイダリー  牝3  55  ルメール  2:26.7     クビ    3.3    1   
 9    10    8   18  エリカエクスプレス  牝3  55  戸崎圭太  2:26.8    3/4   14.5    5   
 10   11    5   10  タイセイプランセス  牝3  55   石橋脩  2:26.9     クビ   42.8    9   
 11   12    2    4     アイサンサン  牝3  55  北村宏司  2:27.0      1  253.7   15   
 12   13    3    6    ビップデイジー  牝3  55 

In [24]:
pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,7,15,カムニャック,牝3,55,シュタル,2:25.7,NaN,14.3,4,474(+4),[西] 友道康夫
1,2,1,1,アルマヴェローチェ,牝3,55,岩田望来,2:25.7,アタマ,3.4,2,490(-6),[西] 上村洋行
2,3,7,13,タガノアビー,牝3,55,藤岡佑介,2:25.9,1.1/4,49.0,10,492(-4),[西] 千田輝彦
3,4,2,3,パラディレーヌ,牝3,55,丹内祐次,2:26.0,クビ,17.7,7,498(0),[西] 千田輝彦
4,5,3,5,リンクスティップ,牝3,55,Ｍ．デム,2:26.2,1.1/4,3.7,3,472(0),[西] 西村真幸
5,6,6,11,ウィルサヴァイブ,牝3,55,団野大成,2:26.4,1.1/4,328.8,18,432(+2),[西] 須貝尚介
6,7,6,12,ブラウンラチェット,牝3,55,レーン,2:26.6,1.1/2,15.3,6,442(-2),[東] 手塚貴久
7,8,8,17,ケリフレッドアスク,牝3,55,ディー,2:26.7,クビ,188.7,14,444(+6),[西] 藤原英昭
8,9,5,9,エンブロイダリー,牝3,55,ルメール,2:26.7,クビ,3.3,1,482(0),[東] 森一誠
9,10,8,18,エリカエクスプレス,牝3,55,戸崎圭太,2:26.8,3/4,14.5,5,456(0),[西] 杉山晴紀


## 開催日一覧を取得

In [25]:
url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
html = urlopen(url).read()
html

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n<!-- block=common__meta_tag_common_race_schedule (d) -->\n<meta http-equiv="content-language" content="ja">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n<meta name="viewport" content="width=1000">\n<meta name="format-detection" content="telephone=no" />\n<title>\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8 | \xa5\xec\xa1\xbc\xa5\xb9\xbe\xf0\xca\xf3(JRA) - netkeiba</title>\n<meta name="description" content="\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8\xa5\xda\xa1\xbc\xa5\xb8\xa4\xc7\xa4\xb9\xa1\xa3\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xe4\xba\xc7\xbf\xb7\xa5\xaa\xa5\xc3\xa5\xba\xa1\xa2\xa5\xec\xa1\xbc\xa5\xb9\xb7\xeb\xb2\xcc\xc2\xae\xca\xf3\xa1\xa2\xca\xa7\xcc\xe1\xbe\xf0\xca\xf3\xa4\xf2\xa4\xcf\xa4\xb8\xa4\xe1\xa1\xa2\xb6\xa5\xc7\xcf\xcd\xbd\xc1\xdb\xa4\

In [26]:
from bs4 import BeautifulSoup # Htmlの細かい要素を取り出したい場合に使うモジュール

soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_common_race_schedule (d) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>開催日程 | レース情報(JRA) - netkeiba</title>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,開催日程,スケジュール,開催日,netkeiba,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba_logo_keiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="開催日程 | レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/top/calendar.html" property="og:url"/>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最

In [27]:
a = soup.find("table", class_="Calendar_Table").find("a") #第二引数指定しないと、デフォルトで最初に見つかった要素が返却される
a

<a href="../top/race_list.html?kaisai_date=20230105" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">5</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">中京</span><span class="JName">京都金杯</span></p>
</div><!-- /. RaceKaisaiBox-->
</a>

In [28]:
a["href"]

'../top/race_list.html?kaisai_date=20230105'

In [29]:
import re #正規表現を使いたい場合のモジュール

re.findall(r"kaisai_date=(\d{8})", a["href"])[0]

'20230105'

In [30]:
a_list = soup.find("table", class_="Calendar_Table").find_all("a")

In [31]:
kaisai_data_list = []
for a in a_list:
    kaisai_data = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
    kaisai_data_list.append(kaisai_data)

In [32]:
kaisai_data_list

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129']

In [33]:
from_ = "2023-01"
to_ = "2023-12"

In [34]:
for date in pd.date_range(from_, to_, freq="MS"):
    year = date.year
    month = date.month
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    print(url)

https://race.netkeiba.com/top/calendar.html?year=2023&month=1
https://race.netkeiba.com/top/calendar.html?year=2023&month=2
https://race.netkeiba.com/top/calendar.html?year=2023&month=3
https://race.netkeiba.com/top/calendar.html?year=2023&month=4
https://race.netkeiba.com/top/calendar.html?year=2023&month=5
https://race.netkeiba.com/top/calendar.html?year=2023&month=6
https://race.netkeiba.com/top/calendar.html?year=2023&month=7
https://race.netkeiba.com/top/calendar.html?year=2023&month=8
https://race.netkeiba.com/top/calendar.html?year=2023&month=9
https://race.netkeiba.com/top/calendar.html?year=2023&month=10
https://race.netkeiba.com/top/calendar.html?year=2023&month=11
https://race.netkeiba.com/top/calendar.html?year=2023&month=12


In [35]:
import time
from tqdm.notebook import tqdm # 実行進捗を可視化できるモジュール

def scrape_kaisai_date(from_, to_):
    kaisai_data_list = []
    for date in tqdm(pd.date_range(from_, to_, freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        html = urlopen(url).read() # スクレイピング
        time.sleep(1) # スクレイピングをFor文使用する際、絶対忘れないように（パフォーマンス低下防ぎ）
        soup = BeautifulSoup(html)
        a_list = soup.find("table", class_="Calendar_Table").find_all("a")
        for a in a_list:
            kaisai_data = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_data_list.append(kaisai_data)
    return kaisai_data_list

In [ ]:
# 例えば
scrape_kaisai_date("2023-01", "2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

## 開催ページからrace_id取得

In [ ]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
html = urlopen(url)
soup = BeautifulSoup(html)

In [ ]:
soup.find("div", class_="RaceList_Box")

BeautifulSoupはjavascriptのような動的に生成オブジェクトを取得できないので、ChromeDriverを使う

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# 最新版のChromeをインストールし、インストール先のパスが返却される
driver_path = ChromeDriverManager().install()
driver_path

'C:\\Users\\saiku\\.wdm\\drivers\\chromedriver\\win64\\140.0.7339.185\\chromedriver-win32/chromedriver.exe'

In [ ]:
# インストールした最新版のChromeブラウザを起動する
driver = webdriver.Chrome(service=Service(driver_path))

In [ ]:
# 指定URLよりページを開く
driver.get(url)

In [ ]:
from selenium.webdriver.common.by import By

li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=140.0.7339.185)
Stacktrace:
	GetHandleVerifier [0x0x810c13+66051]
	GetHandleVerifier [0x0x810c54+66116]
	(No symbol) [0x0x5edb33]
	(No symbol) [0x0x5cc3bd]
	(No symbol) [0x0x6607de]
	(No symbol) [0x0x67b109]
	(No symbol) [0x0x659d86]
	(No symbol) [0x0x62b53e]
	(No symbol) [0x0x62c414]
	GetHandleVerifier [0x0xa58a13+2457603]
	GetHandleVerifier [0x0xa539d2+2437058]
	GetHandleVerifier [0x0x8397f2+232930]
	GetHandleVerifier [0x0x829a18+167944]
	GetHandleVerifier [0x0x83092d+196381]
	GetHandleVerifier [0x0x818ee8+99544]
	GetHandleVerifier [0x0x819082+99954]
	GetHandleVerifier [0x0x80322a+10266]
	BaseThreadInitThunk [0x0x76305d49+25]
	RtlInitializeExceptionChain [0x0x776ad6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x776ad661+561]


In [ ]:
li = li_list[0]
li

<selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.37")>

In [ ]:
href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
href

'https://race.netkeiba.com/race/result.html?race_id=202306010101&rf=race_list'

In [ ]:
import re

re.findall(r"race_id=(\d{12})", href)[0]

'202306010101'

In [ ]:
race_id_list = []
for li in li_list:
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    # print(race_id)
    race_id_list.append(race_id)

In [ ]:
len(race_id_list)

24

In [ ]:
# Chromeブラウザを閉じる
driver.quit()

### 関数化

In [ ]:
import scraping

In [ ]:
kaisai_data_list = scraping.scrape_kaisai_date("2023-01", "2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
import time
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
import traceback

def scrape_race_id_list(kaisai_data_list: list[str]):
    options = Options()
    # バックグラウンドで実行
    options.add_argument("--headless")
    # 最新版のChromeをインストールし、インストール先のパスが返却される
    driver_path = ChromeDriverManager().install()
    race_id_list = []
    
    # インストールした最新版のChromeブラウザを起動する
    with webdriver.Chrome(service=Service(driver_path), options=options) as driver:
        for kaisai_data in tqdm(kaisai_data_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_data}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc())
                break
    return race_id_list
        

In [ ]:
race_id_list = scrape_race_id_list(kaisai_data_list)

  0%|          | 0/107 [00:00<?, ?it/s]

stopped at https://race.netkeiba.com/top/race_list.html?kaisai_date=20230611
Traceback (most recent call last):
  File "C:\Users\saiku\AppData\Local\Temp\ipykernel_17636\1079480709.py", line 23, in scrape_race_id_list
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tools\Python\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 602, in find_element
    return self._execute(Command.FIND_CHILD_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tools\Python\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 573, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tools\Python\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 455, in execute
    response = cast(RemoteConnection, self.command_executor).execute(driver

### スクリプトのチェック

In [ ]:
 # インポートモジュールを自動で同期化
import scraping
%load_ext autoreload

In [ ]:
%autoreload

In [ ]:
race_id_list = scrape_race_id_list(kaisai_data_list[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112',
 '202306010201',
 '202306010202',
 '202306010203',
 '202306010204',
 '202306010205',
 '202306010206',
 '202306010207',
 '202306010208',
 '202306010209',
 '202306010210',
 '202306010211',
 '202306010212',
 '202307010201',
 '202307010202',
 '202307010203',
 '202307010204',
 '202307010205',
 '202307010206',
 '202307010207',
 '202307010208',
 '202307010209',
 '202307010210',
 '202307010211',
 '202307010212',
 '202306010301',
 '202306010302',
 '202306010303',
 '202306010304',
 '202306010305',
 '202306010306',
 '202306010307',
 '202306010308',
 '202306010309',
 '202306010310',
 '202306010311

In [ ]:
from urllib.request import urlopen
url = "https://db.netkeiba.com/race/202505021011"
html = urlopen(url).read()

In [ ]:
import pandas as pd

pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,7,15,カムニャック,牝3,55,シュタル,2:25.7,NaN,14.3,4,474(+4),[西] 友道康夫
1,2,1,1,アルマヴェローチェ,牝3,55,岩田望来,2:25.7,アタマ,3.4,2,490(-6),[西] 上村洋行
2,3,7,13,タガノアビー,牝3,55,藤岡佑介,2:25.9,1.1/4,49.0,10,492(-4),[西] 千田輝彦
3,4,2,3,パラディレーヌ,牝3,55,丹内祐次,2:26.0,クビ,17.7,7,498(0),[西] 千田輝彦
4,5,3,5,リンクスティップ,牝3,55,Ｍ．デム,2:26.2,1.1/4,3.7,3,472(0),[西] 西村真幸
5,6,6,11,ウィルサヴァイブ,牝3,55,団野大成,2:26.4,1.1/4,328.8,18,432(+2),[西] 須貝尚介
6,7,6,12,ブラウンラチェット,牝3,55,レーン,2:26.6,1.1/2,15.3,6,442(-2),[東] 手塚貴久
7,8,8,17,ケリフレッドアスク,牝3,55,ディー,2:26.7,クビ,188.7,14,444(+6),[西] 藤原英昭
8,9,5,9,エンブロイダリー,牝3,55,ルメール,2:26.7,クビ,3.3,1,482(0),[東] 森一誠
9,10,8,18,エリカエクスプレス,牝3,55,戸崎圭太,2:26.8,3/4,14.5,5,456(0),[西] 杉山晴紀


In [ ]:
import pickle

with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [ ]:
len(race_id_list)

3456

In [ ]:
for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    df = pd.read_html(html)[0]
    # print(url)

In [ ]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xa5\xb8\xa5\xe5\xa5\xcb\xa5\xa2\xa5\xab\xa5\xc3\xa5\xd7\xa1\xc32023\xc7\xaf1\xb7\xee5\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\x

In [ ]:
with open("../data/html/202506010101.bin", "wb") as f:
    f.write(html)

In [ ]:
#環境の違いを防げるパス連結
from pathlib import Path

HTML_DIR = Path("..", "data", "html")
HTML_DIR

WindowsPath('../data/html')

In [ ]:
HTML_DIR / "202506010101.bin"

WindowsPath('../data/html/202506010101.bin')

In [ ]:
with open(HTML_DIR / "202506010101.bin", "wb") as f:
    f.write(html)

In [ ]:
race_id_list[:10]

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110']

In [ ]:
import time

for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    time.sleep(1)
    with open(HTML_DIR / f"{race_id}.bin", "wb") as f:
        f.write(html)


In [ ]:
from tqdm.notebook import tqdm
def scrap_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin"
        # binファイルが既に存在する場合はスキップする
        if filepath.is_file():
            print(f"skipped:{race_id}")
            continue
        url = f"https://db.netkeiba.com/race/{race_id}"
        html = urlopen(url).read()
        time.sleep(1)
        with open(filepath, "wb") as f:
            f.write(html)
        html_path_list.append(filepath)
    return html_path_list

In [ ]:
html_path_list = scrap_html_race(race_id_list[:14], save_dir=HTML_DIR)

  0%|          | 0/14 [00:00<?, ?it/s]

skipped:202306010101
skipped:202306010102
skipped:202306010103
skipped:202306010104
skipped:202306010105
skipped:202306010106
skipped:202306010107
skipped:202306010108
skipped:202306010109
skipped:202306010110
skipped:202306010111
skipped:202306010112


In [ ]:
html_path_list

[WindowsPath('../data/html/202306010101.bin'),
 WindowsPath('../data/html/202306010102.bin'),
 WindowsPath('../data/html/202306010103.bin'),
 WindowsPath('../data/html/202306010104.bin'),
 WindowsPath('../data/html/202306010105.bin'),
 WindowsPath('../data/html/202306010106.bin'),
 WindowsPath('../data/html/202306010107.bin'),
 WindowsPath('../data/html/202306010108.bin'),
 WindowsPath('../data/html/202306010109.bin'),
 WindowsPath('../data/html/202306010110.bin'),
 WindowsPath('../data/html/202306010111.bin'),
 WindowsPath('../data/html/202306010112.bin')]

In [ ]:
HTML_RACE_DIR = HTML_DIR / "race"
HTML_RACE_DIR

WindowsPath('../data/html/race')

In [ ]:
# globを使って、ファイルパス全件を取得する
html_paths_race = list(HTML_RACE_DIR.glob("*"))
html_paths_race

[WindowsPath('../data/html/race/202305010101.bin'),
 WindowsPath('../data/html/race/202305010102.bin'),
 WindowsPath('../data/html/race/202305010103.bin'),
 WindowsPath('../data/html/race/202305010104.bin'),
 WindowsPath('../data/html/race/202305010105.bin'),
 WindowsPath('../data/html/race/202305010106.bin'),
 WindowsPath('../data/html/race/202305010107.bin'),
 WindowsPath('../data/html/race/202305010108.bin'),
 WindowsPath('../data/html/race/202305010109.bin'),
 WindowsPath('../data/html/race/202305010110.bin'),
 WindowsPath('../data/html/race/202305010111.bin'),
 WindowsPath('../data/html/race/202305010112.bin'),
 WindowsPath('../data/html/race/202305010201.bin'),
 WindowsPath('../data/html/race/202305010202.bin'),
 WindowsPath('../data/html/race/202305010203.bin'),
 WindowsPath('../data/html/race/202305010204.bin'),
 WindowsPath('../data/html/race/202305010205.bin'),
 WindowsPath('../data/html/race/202305010206.bin'),
 WindowsPath('../data/html/race/202305010207.bin'),
 WindowsPath

In [ ]:
len(html_paths_race)

300

In [ ]:
dfs = {}
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        # stemを使って、ファイル名を取得する
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        # indexを付与し、切れ目をわかりやすくする
        df.index = [race_id] * len(df)
        dfs[race_id] = df
# 辞書型のデータを繋げて一つのテーブルする
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"

In [ ]:
dfs['202305010101']

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
202305010101,1,7,14,メイデンタワー,牝3,54,横山武史,1:26.1,NaN,1.6,1,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54,津村明秀,1:26.8,4,6.0,3,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54,嶋田純次,1:27.1,2,92.7,10,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54,内田博幸,1:27.3,3/4,13.6,4,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52,小林脩斗,1:27.9,3.1/2,5.8,2,450(+2),[東] 奥平雅士
202305010101,6,4,8,スピリチュアル,牝3,54,武藤雅,1:28.0,1/2,25.7,7,444(-6),[東] 武藤善則
202305010101,7,6,11,マラマプア,牝3,53,永野猛蔵,1:28.1,1,15.2,5,482(+2),[東] 伊藤圭三
202305010101,8,2,4,フジジュンフェイス,牝3,54,杉原誠人,1:28.5,2.1/2,108.1,11,442(-4),[東] 伊藤大士
202305010101,9,4,7,テイエムガーネット,牝3,54,武士沢友,1:28.6,1/2,22.6,6,468(+4),[東] 石栗龍彦
202305010101,10,3,6,カノンチャン,牝3,52,横山琉人,1:29.0,2.1/2,134.4,12,446(-4),[東] 高柳瑞樹


In [ ]:
['202305010101'] * 4

['202305010101', '202305010101', '202305010101', '202305010101']

In [ ]:
['202305010101'] * len(dfs)

['202305010101', '202305010101', '202305010101']

In [ ]:
# 辞書型のデータを繋げて一つのテーブルする
pd.concat(dfs.values())

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
202305010101,1,7,14,メイデンタワー,牝3,54,横山武史,1:26.1,NaN,1.6,1,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54,津村明秀,1:26.8,4,6.0,3,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54,嶋田純次,1:27.1,2,92.7,10,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54,内田博幸,1:27.3,3/4,13.6,4,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52,小林脩斗,1:27.9,3.1/2,5.8,2,450(+2),[東] 奥平雅士
202305010101,6,4,8,スピリチュアル,牝3,54,武藤雅,1:28.0,1/2,25.7,7,444(-6),[東] 武藤善則
202305010101,7,6,11,マラマプア,牝3,53,永野猛蔵,1:28.1,1,15.2,5,482(+2),[東] 伊藤圭三
202305010101,8,2,4,フジジュンフェイス,牝3,54,杉原誠人,1:28.5,2.1/2,108.1,11,442(-4),[東] 伊藤大士
202305010101,9,4,7,テイエムガーネット,牝3,54,武士沢友,1:28.6,1/2,22.6,6,468(+4),[東] 石栗龍彦
202305010101,10,3,6,カノンチャン,牝3,52,横山琉人,1:29.0,2.1/2,134.4,12,446(-4),[東] 高柳瑞樹


In [ ]:
from tqdm.notebook import tqdm
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, "rb") as f:
            # stemを使って、ファイル名を取得する
            race_id = html_path.stem
            html = f.read()
            df = pd.read_html(html)[0]
            # indexを付与し、切れ目をわかりやすくする
            df.index = [race_id] * len(df)
            dfs[race_id] = df
    # 辞書型のデータを繋げて一つのテーブルする
    concat_df = pd.concat(dfs.values())
    concat_df.index.name = "race_id"
    return concat_df

In [ ]:
results = create_results(html_path_list=html_path_list)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202305010101,1,7,14,メイデンタワー,牝3,54.0,横山武史,1:26.1,NaN,1.6,1.0,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54.0,津村明秀,1:26.8,4,6.0,3.0,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54.0,嶋田純次,1:27.1,2,92.7,10.0,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54.0,内田博幸,1:27.3,3/4,13.6,4.0,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52.0,小林脩斗,1:27.9,3.1/2,5.8,2.0,450(+2),[東] 奥平雅士
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310010512,14,5,10,スウィートメリナ,牝6,54.0,小林凌大,1:10.7,ハナ,108.4,18.0,450(+8),[西] 羽月友彦
202310010512,15,1,1,トーホウデュラン,牡4,58.0,西村淳也,1:11.0,2,4.9,1.0,488(+4),[西] 石坂公一
202310010512,16,2,3,ピンクセイラー,牝4,55.0,秋山稔樹,1:11.4,2,33.2,10.0,460(+14),[東] 小笠倫弘


### 今回のテーマ
1. 馬の過去成績テーブルの作成
2. Pythonによるファイルの書き込み・読み取り

In [1]:
url = "https://db.netkeiba.com/horse/result/2020103575/"

In [2]:
from urllib.request import urlopen
html = urlopen(url).read()
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n    <head>\n\t\n\t<!-- block=meta_tag_basic (d) -->\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?20210819" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/db_detail.css?20180621" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/horse_detail.css?20250620" rel="stylesheet" t

In [3]:
import pandas as pd

pd.read_html(html)[0]

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2025/08/17,3中京8,晴,6,大垣特別(2勝クラス),NaN,16,3,6,219.8,...,2.5,**,10-10,35.4-36.5,38.3,500(-6),NaN,NaN,タイセイレスポンス,NaN
1,2025/06/22,3東京6,晴,9,甲州街道特別(2勝クラス),NaN,16,5,9,130.9,...,3.7,**,2-3,34.8-37.0,40.4,506(+2),NaN,NaN,トクシーカイザー,NaN
2,2025/03/30,2中京6,晴,12,鈴鹿特別(2勝クラス),NaN,16,8,16,137.3,...,1.2,**,5-4,35.3-37.2,37.9,504(-4),NaN,NaN,セミマル,NaN
3,2024/12/22,5中山8,晴,8,冬至特別(2勝クラス),NaN,16,1,2,99.0,...,1.4,**,11-11,33.6-38.0,38.4,508(+12),NaN,NaN,ロードフロンティア,NaN
4,2024/08/31,2中京7,小雨,10,鳥羽特別(2勝クラス),NaN,15,3,5,32.6,...,4.6,**,10-10,34.7-36.8,40.5,496(-8),NaN,NaN,カズゴルティス,NaN
5,2024/07/27,2新潟1,曇,12,出雲崎特別(2勝クラス),NaN,15,6,10,8.0,...,0.9,**,4-4,34.2-36.7,37.3,504(+10),NaN,NaN,エランティス,NaN
6,2024/05/19,1新潟8,晴,10,火打山特別(2勝クラス),NaN,15,5,9,13.0,...,0.6,**,6-6,33.6-37.5,37.1,494(-6),NaN,NaN,シラキヌ,230.0
7,2024/05/05,1新潟4,晴,12,4歳以上1勝クラス,NaN,13,4,4,4.5,...,-0.2,**,7-7,35.2-37.6,36.9,500(+2),NaN,NaN,(セキテイオー),800.0
8,2024/02/25,1阪神2,小雨,12,4歳以上1勝クラス,NaN,15,7,13,8.6,...,0.9,**,8-10,34.6-36.4,36.3,498(0),NaN,NaN,サクハル,NaN
9,2023/12/16,5中山5,晴,12,3歳以上1勝クラス,NaN,16,4,7,4.1,...,1.4,**,2-3,34.7-36.5,37.7,498(+4),NaN,NaN,ヤングワールド,NaN


In [4]:
url = "https://db.netkeiba.com/race/202306010101"
html = urlopen(url).read()

In [5]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>3\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32023\xc7\xaf1\xb7\xee5\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "lxml")
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>3歳未勝利｜2023年1月5日 | 競馬データベース - netkeiba</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手・調教師・馬主・生産者の全データがご覧いただけます。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="競馬データベース | 競走馬・騎手など情報満載 - netkeiba" property="og:title"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手

In [7]:
table = soup.find("table", class_="race_table_01 nk_tb_common")
table

<table cellpadding="0" cellspacing="1" class="race_table_01 nk_tb_common" summary="レース結果">
<tr class="txt_c">
<th nowrap="nowrap">着<br/>順</th>
<th nowrap="nowrap">枠<br/>番</th>
<th nowrap="nowrap">馬<br/>番</th>
<th nowrap="nowrap">馬名</th>
<th nowrap="nowrap">性齢</th>
<th nowrap="nowrap">斤量</th>
<th nowrap="nowrap">騎手</th>
<th nowrap="nowrap">タイム</th>
<th class="" nowrap="nowrap">着差</th>
<diary_snap_cut>
<th class="" nowrap="nowrap">ﾀｲﾑ<br/>指数<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th class="" nowrap="nowrap">通過</th>
<th class="" nowrap="nowrap">上り</th>
</diary_snap_cut>
<th nowrap="nowrap">単勝</th>
<th nowrap="nowrap">人<br/>気</th>
<th nowrap="nowrap">馬体重</th>
<diary_snap_cut>
<th class="" nowrap="nowrap">調教<br/>ﾀｲﾑ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th class="" nowrap="nowrap">厩舎<br/>ｺﾒﾝﾄ<br/><img height="16" src="https://cdn.ne

In [8]:
table.find_all("a", href="/horse/2020103575/")

[<a href="/horse/2020103575/" id="umalink_202306010101" title="シュバルツガイスト">シュバルツガイスト</a>]

In [9]:
import re

# /horse/から始まるhref属性を持つaタグの一覧を取得する
a_list = table.find_all("a", href=re.compile(r"^/horse/"))
a_list

[<a href="/horse/2020103575/" id="umalink_202306010101" title="シュバルツガイスト">シュバルツガイスト</a>,
 <a href="/horse/2020107073/" id="umalink_202306010101" title="ジェイケイファイン">ジェイケイファイン</a>,
 <a href="/horse/2020102562/" id="umalink_202306010101" title="レディアス">レディアス</a>,
 <a href="/horse/2020106345/" id="umalink_202306010101" title="ハローマイラブリー">ハローマイラブリー</a>,
 <a href="/horse/2020100039/" id="umalink_202306010101" title="ベアカワイコチャン">ベアカワイコチャン</a>,
 <a href="/horse/2020104817/" id="umalink_202306010101" title="ラディーヴァ">ラディーヴァ</a>,
 <a href="/horse/2020104295/" id="umalink_202306010101" title="メルティーショコラ">メルティーショコラ</a>,
 <a href="/horse/2020105491/" id="umalink_202306010101" title="ニシノスイセイ">ニシノスイセイ</a>,
 <a href="/horse/2020104358/" id="umalink_202306010101" title="タッカーカムカム">タッカーカムカム</a>,
 <a href="/horse/2020105324/" id="umalink_202306010101" title="ペイシャコリン">ペイシャコリン</a>,
 <a href="/horse/2020103960/" id="umalink_202306010101" title="オメガキャプテン">オメガキャプテン</a>,
 <a href="/horse/2020102110/" id="umalink_20230

In [10]:
a = a_list[0]
a

<a href="/horse/2020103575/" id="umalink_202306010101" title="シュバルツガイスト">シュバルツガイスト</a>

In [11]:
re.findall(r"\d{10}", a["href"])[0]

'2020103575'

In [12]:
horse_id_list = []
for a in a_list:
    horse_id = re.findall(r"\d{10}", a["href"])[0]
    # print(horse_id)
    horse_id_list.append(horse_id)

In [13]:
len(horse_id_list)

16

In [5]:
import create_rawdf
from pathlib import Path
%reload_ext autoreload

In [ ]:
# html_paths_raceが失われた場合はファイルから読み込む
if "html_paths_race" not in locals() or len(html_paths_race) == 0:
    # globを使って、binファイルパス全件を取得する
    html_paths_race = list(scraping.HTML_RACE_DIR.glob("*.bin"))
html_paths_race

In [8]:
results = create_rawdf.create_results(html_paths_race[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
results.columns = results.columns.str.replace(" ","")

In [10]:
results.columns

Index(['着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気',
       '馬体重', '調教師', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id'],
      dtype='object')

In [11]:
save_dir = Path("..", "data", "rawdf")

In [12]:
results.to_csv(save_dir / "results.csv")

In [13]:
horse_id_list = results["horse_id"].unique()

In [14]:
len(horse_id_list)

149

In [15]:
import scraping
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
html_paths_horse = scraping.scrap_html_horse(horse_id_list=horse_id_list)

  0%|          | 0/149 [00:00<?, ?it/s]

In [17]:
len(html_paths_horse)

149

In [18]:
%autoreload

In [19]:
horse_results = create_rawdf.create_horse_results(html_path_list=html_paths_horse)

  0%|          | 0/149 [00:00<?, ?it/s]

            日付     開催 天 気   R         レース名  映 像  頭 数  枠 番  馬 番  オ ッ ズ  ...  \
0   2025/06/01  2京都12   晴   8    4歳以上2勝クラス  NaN   16    7   14   13.6  ...   
1   2025/04/12   3中山5   晴  12    4歳以上2勝クラス  NaN   16    3    5   12.6  ...   
2   2025/03/02   2中山2   晴  12    4歳以上2勝クラス  NaN   16    7   14   14.7  ...   
3   2024/12/14   5中山5   晴  12    3歳以上2勝クラス  NaN   16    5    9   13.4  ...   
4   2024/10/06   5京都2   晴  12    3歳以上2勝クラス  NaN   15    3    5   14.2  ...   
5   2024/08/11   2札幌2   晴   9  桑園特別(2勝クラス)  NaN   12    8   11    5.8  ...   
6   2024/04/28   2東京4   晴  12    4歳以上2勝クラス  NaN   16    8   16    4.6  ...   
7   2024/02/24   2中山1   晴  12    4歳以上2勝クラス  NaN   16    8   15    4.8  ...   
8   2023/12/17   5阪神6   晴   8    3歳以上2勝クラス  NaN   14    2    2    4.7  ...   
9   2023/10/08   4東京2  小雨  12    3歳以上2勝クラス  NaN   16    2    3    3.0  ...   
10  2023/06/03   3東京1   晴  12    3歳以上1勝クラス  NaN   16    3    5    3.2  ...   
11  2023/05/13   2東京7  小雨   6      3歳1勝クラス  NaN   16    1    1  

# レース情報の取得

In [1]:
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
import re

DATA_DIR = Path("..", "data")
HTML_DIR = DATA_DIR / "html"
HTML_RACE_DIR = HTML_DIR / "race"


In [3]:
html_path_list = list(HTML_RACE_DIR.glob("*.bin"))

In [4]:
html_path_list[0]

WindowsPath('../data/html/race/202301010101.bin')

In [5]:
with open(html_path_list[0], "rb") as f:
    html = f.read()

In [6]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>2\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32023\xc7\xaf7\xb7\xee22\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1

In [7]:
soup = BeautifulSoup(html, "lxml")

In [12]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>2歳未勝利｜2023年7月22日 | 競馬データベース - netkeiba</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手・調教師・馬主・生産者の全データがご覧いただけます。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="競馬データベース | 競走馬・騎手など情報満載 - netkeiba" property="og:title"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎

In [8]:
soup_info = soup.find("div", class_="data_intro")
soup_info

<div class="data_intro">
<dl class="racedata fc">
<dt>
1 R
</dt>
<dd>
<h1>2歳未勝利<!--img src="style/netkeiba.ja/image/race_grade_g2_01.png" alt=""--></h1>
<p>
<diary_snap_cut>
<span>
芝右1200m / 
天候 : 晴 / 
芝 : 良  / 
発走 : 09:50
</span><br/>
<diary_snap_cut>
</diary_snap_cut>
</diary_snap_cut></p>
</dd>
</dl>
<p class="smalltxt">2023年07月22日 1回札幌1日目 2歳未勝利  (混)[指](馬齢)</p>
<diary_snap_cut>
<ul class="btn_link_list fc">
<li class="race_btn_MOVIE end">
<a href="https://db.netkeiba.com/race/movie/202301010101"><img alt="レース映像" class="imgover" src="https://cdn.netkeiba.com/img.db/common/image//race_btn_movie_on.png"/></a>
</li>
</ul>
</diary_snap_cut>
</div>

In [9]:
soup_info.find("h1").text

'2歳未勝利'

In [15]:
tmp = soup_info.find("p").text.replace(" ", "")
tmp

'\n\n\n芝右1200m\xa0/\xa0\n天候:晴\xa0/\xa0\n芝:良\xa0\xa0/\xa0\n発走:09:50\n\n\n\n'

In [16]:
# \w普通の文字(半角スペース、記号などが除外される) []にすると括弧内の文字を対象となる
re.findall(r"[\w:]+", tmp)

['芝右1200m', '天候:晴', '芝:良', '発走:09:50']

In [15]:
tmp = soup_info.find_all("p")[1].text
tmp

'2023年07月22日 1回札幌1日目 2歳未勝利\xa0\xa0(混)[指](馬齢)'

In [20]:
tmp1 = re.findall(r"\w+", tmp)
tmp1

['2023年07月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']

In [58]:
info_dict = {}
info_dict["title"] = soup_info.find("h1").text
info_dict["info1"] = re.findall(r"[\w:]+", soup_info.find("p").text.replace(" ", ""))
info_dict["info2"] = re.findall(r"\w+", soup_info.find_all("p")[1].text)
info_dict

{'title': '2歳未勝利',
 'info1': ['芝右1200m', '天候:晴', '芝:良', '発走:09:50'],
 'info2': ['2023年07月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']}

In [ ]:
# .Tは行列を入れ替え
pd.DataFrame().from_dict(info_dict, orient="index").T

,title,info1,info2
0,2歳未勝利,"[芝右1200m, 天候:晴, 芝:良, 発走:09:50]","[2023年07月22日, 1回札幌1日目, 2歳未勝利, 混, 指, 馬齢]"


In [27]:
import create_rawdf

race_info = create_rawdf.create_race_info(html_path_list)

  0%|          | 0/3456 [00:00<?, ?it/s]

In [28]:
race_info

,title,info1,info2
race_id,,,
202301010101,2歳未勝利,"[芝右1200m, 天候:晴, 芝:良, 発走:09:50]","[2023年07月22日, 1回札幌1日目, 2歳未勝利, 混, 指, 馬齢]"
202301010102,3歳未勝利,"[ダ右1000m, 天候:晴, ダート:良, 発走:10:25]","[2023年07月22日, 1回札幌1日目, 3歳未勝利, 指, 馬齢]"
202301010103,3歳未勝利,"[ダ右1700m, 天候:晴, ダート:良, 発走:10:55]","[2023年07月22日, 1回札幌1日目, 3歳未勝利, 指, 馬齢]"
202301010104,3歳未勝利,"[芝右1500m, 天候:晴, 芝:良, 発走:11:25]","[2023年07月22日, 1回札幌1日目, 3歳未勝利, 牝, 指, 馬齢]"
202301010105,2歳新馬,"[ダ右1700m, 天候:晴, ダート:良, 発走:12:15]","[2023年07月22日, 1回札幌1日目, 2歳新馬, 混, 指, 馬齢]"
...,...,...,...
202310030808,3歳以上1勝クラス,"[ダ右1000m, 天候:晴, ダート:良, 発走:13:55]","[2023年09月03日, 3回小倉8日目, 3歳以上1勝クラス, 指, 定量]"
202310030809,西日本新聞杯(2勝),"[芝右1200m, 天候:晴, 芝:良, 発走:14:25, 過去の西日本新聞杯]","[2023年09月03日, 3回小倉8日目, 3歳以上2勝クラス, 混, 指, 定量]"
202310030810,宮崎ステークス(3勝),"[ダ右1700m, 天候:晴, ダート:良, 発走:15:01, 過去の宮崎ステークス]","[2023年09月03日, 3回小倉8日目, 3歳以上3勝クラス, 混, 特指, 定量]"


In [2]:
df = pd.read_csv(DATA_DIR / "rawdf" / "race_info.csv", sep="\t")
df

,race_id,title,info1,info2
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年07月22日', '1回札幌1日目', '2歳未勝利', '混', '指', ..."
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年07月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年07月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年07月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', ..."
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年07月22日', '1回札幌1日目', '2歳新馬', '混', '指', '..."
...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年09月03日', '3回小倉8日目', '3歳以上1勝クラス', '指', '..."
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年09月03日', '3回小倉8日目', '3歳以上2勝クラス', '混', '..."
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年09月03日', '3回小倉8日目', '3歳以上3勝クラス', '混', '..."
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年09月03日', '3回小倉8日目', '2歳オープン', '国際', '特指..."


In [10]:
df["info2"].map(lambda x: eval(x)[0])

0       2023年07月22日
1       2023年07月22日
2       2023年07月22日
3       2023年07月22日
4       2023年07月22日
           ...     
3451    2023年09月03日
3452    2023年09月03日
3453    2023年09月03日
3454    2023年09月03日
3455    2023年09月03日
Name: info2, Length: 3456, dtype: object

In [16]:
df["info1"].str.extract(r"(\d+)")

,0
0,1200
1,1000
2,1700
3,1500
4,1700
...,...
3451,1000
3452,1200
3453,1700
3454,1200


In [12]:
df["info1"].str[2]

0       芝
1       ダ
2       ダ
3       芝
4       ダ
       ..
3451    ダ
3452    芝
3453    ダ
3454    芝
3455    芝
Name: info1, Length: 3456, dtype: object

In [3]:
df["info1"].str.extract(r"\天候:(\w+)")[0]

0       晴
1       晴
2       晴
3       晴
4       晴
       ..
3451    晴
3452    晴
3453    晴
3454    晴
3455    晴
Name: 0, Length: 3456, dtype: object

In [6]:
df["info1"].str.extract(r"(芝|ダート|障害):(\w+)")[1]

,0,1
0,芝,良
1,ダート,良
2,ダート,良
3,芝,良
4,ダート,良
...,...,...
3451,ダート,良
3452,芝,良
3453,ダート,良
3454,芝,良


In [26]:
df["tmp"] = df["info1"].map(lambda x: eval(x)[0])
df["tmp"]

0       芝右1200m
1       ダ右1000m
2       ダ右1700m
3       芝右1500m
4       ダ右1700m
         ...   
3451    ダ右1000m
3452    芝右1200m
3453    ダ右1700m
3454    芝右1200m
3455    芝右2600m
Name: tmp, Length: 3456, dtype: object

In [30]:
df["tmp"].str.extract(r"(\d+)").value_counts()

0   
1800    799
1200    631
1400    440
1600    425
2000    368
1700    205
1000     86
2400     77
2200     70
1900     44
2600     39
2100     38
1150     37
1300     24
1500     21
2860     21
2500     15
3000     14
2750     12
2970     10
2880     10
3110      8
2770      6
3390      5
2910      3
3380      3
2300      2
3200      2
3140      2
2         1
3300      1
3210      1
3170      1
3350      1
3330      1
3400      1
3570      1
3900      1
3930      1
4100      1
4250      1
Name: count, dtype: int64

In [101]:
df

,race_id,title,info1,info2,date
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年07月22日', '1回札幌1日目', '2歳未勝利', '混', '指', ...",2023年07月22日
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年07月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']",2023年07月22日
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年07月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']",2023年07月22日
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年07月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', ...",2023年07月22日
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年07月22日', '1回札幌1日目', '2歳新馬', '混', '指', '...",2023年07月22日
...,...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年09月03日', '3回小倉8日目', '3歳以上1勝クラス', '指', '...",2023年07月22日
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年09月03日', '3回小倉8日目', '3歳以上2勝クラス', '混', '...",2023年07月22日
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年09月03日', '3回小倉8日目', '3歳以上3勝クラス', '混', '...",2023年07月22日
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年09月03日', '3回小倉8日目', '2歳オープン', '国際', '特指...",2023年07月22日


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3456 entries, 0 to 3455
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   race_id  3456 non-null   int64 
 1   title    3456 non-null   object
 2   info1    3456 non-null   object
 3   info2    3456 non-null   object
 4   date     3456 non-null   object
dtypes: int64(1), object(4)
memory usage: 135.1+ KB


In [ ]:
pd.to_date(race_info["info2"], errors="coerce")